In [ ]:
import healpy as hp
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [ ]:
kSZ_PS = np.load('./lensedkSZ_NS_2560_R_2968_P_1536_DV_192.npy')
overdensity_PS = np.load('./overdensity_NS_2560_R_2968_P_1536_DV_192.npy')

In [ ]:
import camb
import sys, argparse, multiprocessing
from scipy.signal import savgol_filter

#Make Fake CMB
h=0.69
pars = camb.CAMBparams()
pars.set_cosmology(H0=100.0*h, ombh2=0.048*h**2, omch2=0.262*h**2, mnu=0.06, omk=0)
pars.InitPower.set_params(As=2e-9, ns=0.96, r=0)
pars.set_for_lmax(6144, lens_potential_accuracy=0)
results = camb.get_results(pars)
powers =results.get_cmb_power_spectra(pars, CMB_unit='K')
l=np.arange(0,len(powers['total'][:,0]))
cambFactor = l*(l+1)/(2*np.pi)
CMB_camb = powers['total'][:,0]/cambFactor
CMB_camb[0]=0.0


In [ ]:
plt.figure(dpi=800)
plt.loglog(kSZ_PS);
plt.loglog(CMB_camb);
plt.legend(['kSZ','cmb']);

In [ ]:
NSIDE=512

In [ ]:
fakekSZ = hp.synfast(kSZ_PS,nside=NSIDE)
fakeCMB = hp.synfast(CMB_camb,nside=NSIDE)

In [ ]:
hp.mollview(fakekSZ)

In [ ]:
hp.mollview(fakeCMB)

In [ ]:
hp.mollview(fakeCMB,nest=True)

## Generate squares of data for ML

In [ ]:
NSIDE=2048

In [ ]:
def r_square(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
genMaps = 20
nsideDivs = 16
angularDivs = 12*nsideDivs**2
npix = 12*NSIDE**2
block_size = npix//angularDivs

CMB_squares = np.zeros((genMaps,angularDivs,block_size))
kSZ_squares = np.zeros((genMaps,angularDivs,block_size))

#totalPix = 0

for i in range(genMaps):
    fakekSZ = hp.reorder(hp.synfast(kSZ_PS,nside=NSIDE),r2n=True)
    fakeCMB = hp.reorder(hp.synfast(CMB_camb,nside=NSIDE),r2n=True)
    for j in range(angularDivs):
        pixels = np.arange(j*block_size,(j+1)*block_size)
        kSZ_squares[i,j,:] = fakekSZ[pixels]
        CMB_squares[i,j,:] = fakeCMB[pixels]
        #fakeCMB[pixels]=j

In [ ]:
plt.loglog(kSZ_PS)
plt.loglog(CMB_camb)
plt.loglog(hp.anafast(hp.reorder(fakekSZ,n2r=True)))
plt.loglog(hp.anafast(hp.reorder(fakeCMB,n2r=True)))
plt.legend(['kSZ','CMB','gen kSZ','gen CMB'])

In [ ]:
x_raw = (kSZ_squares+CMB_squares).reshape(-1, block_size)
y_raw = kSZ_squares.reshape(-1, block_size)

numSets = genMaps*angularDivs
x_train, x_test = np.split(x_raw, indices_or_sections=[numSets-numSets//genMaps])
y_train, y_test = np.split(y_raw, indices_or_sections=[numSets-numSets//genMaps])

In [ ]:
dset_test = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(x_test),
                                 tf.data.Dataset.from_tensor_slices(y_test)))
dset_test = dset_test.shuffle(3000)
dset_test = dset_test.batch(250)

dset_train = tf.data.Dataset.zip((tf.data.Dataset.from_tensor_slices(x_train),
                                  tf.data.Dataset.from_tensor_slices(y_train)))
dset_train = dset_train.shuffle(3000)
dset_train = dset_train.batch(250)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

dset_train = dset_train.prefetch(buffer_size=AUTOTUNE)
dset_test = dset_test.prefetch(buffer_size=AUTOTUNE)

In [ ]:
block_size

In [ ]:
# simple dense model
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(block_size,)))
model.add(tf.keras.layers.Dense(block_size, activation='tanh'))
model.add(tf.keras.layers.Dense(block_size,activation="tanh"))
#model.add(tf.keras.layers.Dense(3*block_size,activation="linear"))
#model.add(tf.keras.layers.Dense(2*block_size,activation="linear"))
#model.add(tf.keras.layers.Dense(2*block_size,activation="tanh"))
model.add(tf.keras.layers.Dense(block_size,activation="linear"))
#model.add(tf.keras.layers.Dense(2*block_size,activation="tanh"))
model.add(tf.keras.layers.Dense(block_size,activation="tanh"))
model.output_shape

In [ ]:
side = int(np.sqrt(block_size))
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Reshape((side, -1,1), input_shape=(block_size,)))
model.add(tf.keras.layers.Conv2D(side, (8,8), activation="sigmoid"))
model.add(tf.keras.layers.Conv2D(side, (4,4), activation="tanh"))
model.add(tf.keras.layers.Conv2D(side, (4,4), activation="tanh"))
model.add(tf.keras.layers.Conv2D(side, (4,4), activation="tanh"))
#model.add(tf.keras.layers.Conv2D(16, (16,16), activation="sigmoid"))
#model.add(tf.keras.layers.Conv2D(1, (8,8), activation="sigmoid"))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(block_size,activation="tanh"))
#model.add(tf.keras.layers.Dense(block_size,activation="sigmoid"))
model.add(tf.keras.layers.Dense(block_size,activation="linear"))
model.output_shape

In [ ]:
model.summary()

In [ ]:
#opt = tf.keras.optimizers.AdamW(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07)
opt = tf.keras.optimizers.Adadelta(learning_rate=0.01)
#opt = tf.keras.optimizers.Adagrad()

earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = tf.keras.callbacks.ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=25, verbose=1, epsilon=1e-7, mode='min')

model.compile(optimizer=opt,
              #loss=tf.keras.losses.Huber(delta=1.0,reduction="auto",name="huber_loss"),
              #loss="MSE",
              loss = "MAE",
              #loss = 
              #loss = 'cosine_similarity',
              metrics=[r_square]
)

In [ ]:
initial_epochs = 100
history = model.fit(
    dset_train,
    epochs=initial_epochs,
    validation_data = dset_test,
    callbacks=[earlyStopping, mcp_save, reduce_lr_loss]
)

In [ ]:
model_loaded = tf.keras.models.load_model('.mdl_wts.hdf5',custom_objects={'r_square':r_square})

In [ ]:
fakekSZTest = hp.reorder(hp.synfast(kSZ_PS,nside=NSIDE),r2n=True) #nest ordered
fakeCMBTest = hp.reorder(hp.synfast(CMB_camb,nside=NSIDE),r2n=True) #nest ordered

generatedInput = fakekSZTest + fakeCMBTest
generatedInput = generatedInput.reshape(angularDivs,block_size)

modeledkSZ = model_loaded.predict(generatedInput) #nest ordered

modeledkSZ = modeledkSZ.flatten()

# for j in range(angularDivs):
#     pixels = np.arange(j*block_size,(j+1)*block_size)
#     modeledkSZ[j,:] = model_loaded.predict(fakekSZTest[pixels].reshape(1,-1)) #extracting nest ordered chunks
#     if j%25 == 0:
#         print(j)
    
modeledkSZ = hp.reorder(modeledkSZ,n2r=True) #ring ordered
fakekSZTest = hp.reorder(fakekSZTest,n2r=True) #ring ordered
generatedInput = hp.reorder(fakekSZTest + fakeCMBTest,n2r=True) #ring ordered

In [ ]:
hp.mollview(modeledkSZ)#,nest=False)#,min=-15*10**-6,max=15*10**-6)

In [ ]:
hp.mollview(fakekSZTest)#,nest=True)#,min=-5*10**-6,max=5*10**-6)

In [ ]:
hp.mollview(fakekSZTest/generatedInput)#,nest=True)#,min=-5*10**-6,max=5*10**-6)

In [ ]:
plt.hist(fakekSZTest/generatedInput,bins=np.linspace(-.1,.1));

In [ ]:
plt.imshow(np.reshape(generatedInput[0:block_size],(int(np.sqrt(block_size)),-1)));
plt.colorbar();
plt.title('Model Input');

In [ ]:
plt.imshow(np.reshape(modeledkSZ[0:block_size],(int(np.sqrt(block_size)),-1)));
plt.colorbar();
plt.title('predicted kSZ');

In [ ]:
plt.imshow(np.reshape(fakekSZTest[0:block_size],(int(np.sqrt(block_size)),-1)));
plt.colorbar();
plt.title('Actual kSZ');

In [ ]:
plt.imshow(np.reshape(fakekSZTest[0:block_size]-modeledkSZ[0:block_size],(int(np.sqrt(block_size)),-1)));
plt.colorbar();
plt.title('Difference');

In [ ]:
truthPS = hp.anafast(fakekSZTest)
predictedPS = hp.anafast(modeledkSZ)
crossCorrelation = hp.anafast(fakekSZTest,modeledkSZ)

In [ ]:
plt.loglog(truthPS);
plt.loglog(predictedPS);
plt.legend(['Truth','Prediction']);

In [ ]:
plt.semilogx(crossCorrelation/np.sqrt(np.multiply(truthPS,predictedPS)));
plt.axhline(y = 0, color = 'r', linestyle = '-');